In [1]:
import geopandas as gpd
import rasterio
from rasterio.windows import Window
import matplotlib.pyplot as plt
import numpy as np

In [5]:

shapefile_path = "/home/sambal/Desktop/Plaksha-CDA/change_detection/data/modis/DL_FIRE_M-C61_360733/fire_archive_M-C61_360733.shp"
points = gpd.read_file(shapefile_path)

In [11]:
points

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,TYPE,geometry
0,32.3088,72.6911,318.2,1.0,1.0,2022-09-15,0545,Terra,MODIS,61,6.03,299.7,7.5,D,0,POINT (282620.737 3577006.469)
1,30.0867,74.0412,321.0,1.1,1.0,2022-09-15,0546,Terra,MODIS,68,6.03,301.7,10.6,D,0,POINT (407604.287 3328780.056)
2,30.2552,74.2629,319.3,1.1,1.0,2022-09-15,0546,Terra,MODIS,27,6.03,300.5,8.9,D,0,POINT (429090.042 3347293.956)
3,30.1110,74.3432,319.0,1.1,1.1,2022-09-15,0546,Terra,MODIS,62,6.03,301.7,9.0,D,0,POINT (436723.083 3331267.110)
4,31.4562,72.4756,335.2,1.1,1.1,2022-09-15,0840,Aqua,MODIS,82,6.03,303.3,28.7,D,0,POINT (260124.780 3482920.868)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14581,31.5819,72.7753,317.8,1.1,1.1,2022-11-30,0840,Aqua,MODIS,72,6.03,297.6,14.5,D,0,POINT (288893.120 3496241.091)
14582,31.5601,70.8812,312.9,1.0,1.0,2022-11-30,0840,Aqua,MODIS,54,6.03,297.7,8.3,D,0,POINT (108958.035 3499042.106)
14583,32.5435,73.7267,306.6,1.3,1.1,2022-11-30,0840,Aqua,MODIS,54,6.03,295.8,5.4,D,0,POINT (380440.689 3601395.884)
14584,33.1638,71.5793,308.8,1.1,1.0,2022-11-30,0841,Aqua,MODIS,55,6.03,296.9,5.5,D,0,POINT (180967.563 3674659.512)


In [8]:
import geopandas as gpd
import rasterio
from rasterio.windows import Window
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Circle
def normalize(img, target_type_min, target_type_max, target_type):
        imin = img.min()
        imax = img.max()
        a = (target_type_max - target_type_min) / (imax - imin)
        b = target_type_max - a * imax
        new_img = (a * img + b).astype(target_type)
        return new_img

shapefile_path = "/home/sambal/Desktop/Plaksha-CDA/change_detection/data/modis/DL_FIRE_M-C61_360733/fire_archive_M-C61_360733.shp"
points = gpd.read_file(shapefile_path)
#points=points[points['ACQ_DATE']=='2022-10-19']

satellite_image_path = "/home/sambal/Desktop/Plaksha-CDA/change_detection/data/satellite_data/sentinel_data/sangrur/images/T43REP_20221019T053829_TCI_10m.tif"
with rasterio.open(satellite_image_path) as src:
    image_extent = src.bounds

# Ensure both datasets have the same CRS
points = points.to_crs(src.crs)

# Determine the intersecting points based on the extent
intersecting_points = points.cx[image_extent[0]:image_extent[2], image_extent[1]:image_extent[3]]


In [9]:
intersecting_points

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,TYPE,geometry
16,30.2607,75.2336,315.8,1.0,1.0,2022-09-19,0814,Aqua,MODIS,7,6.03,301.2,6.7,D,2,POINT (522471.088 3347696.683)
182,29.8548,75.1942,323.9,1.2,1.1,2022-10-05,0810,Aqua,MODIS,70,6.03,306.3,14.1,D,0,POINT (518757.365 3302712.018)
185,30.2336,76.0352,313.0,1.1,1.0,2022-10-05,0810,Aqua,MODIS,49,6.03,302.0,5.5,D,0,POINT (599610.668 3345123.723)
249,29.7924,76.0634,317.6,1.1,1.0,2022-10-05,0810,Aqua,MODIS,63,6.03,302.5,10.0,D,0,POINT (602778.278 3296255.879)
251,29.8447,75.3950,320.3,1.1,1.1,2022-10-05,0810,Aqua,MODIS,64,6.03,304.2,12.9,D,0,POINT (538156.168 3301642.512)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14344,29.8004,75.5144,317.8,1.1,1.0,2022-11-24,0537,Terra,MODIS,70,6.03,301.8,9.6,D,0,POINT (549712.013 3296779.297)
14345,29.7986,75.5254,317.4,1.1,1.0,2022-11-24,0537,Terra,MODIS,70,6.03,301.5,9.4,D,0,POINT (550775.986 3296584.638)
14375,29.9403,76.1177,325.1,1.0,1.0,2022-11-25,0823,Aqua,MODIS,78,6.03,302.4,17.5,D,0,POINT (607867.502 3312695.289)
14535,30.4310,75.2436,315.0,2.4,1.5,2022-11-28,0853,Aqua,MODIS,69,6.03,301.7,25.3,D,0,POINT (523392.502 3366570.401)


In [6]:
intersecting_points.shape

(21, 16)